In [28]:
# postServer-mqtt client
import paho.mqtt.client as mqtt
from bigchaindb_driver.crypto import generate_keypair
import json
from bigchaindb_driver import BigchainDB
bdb_root_url = 'http://192.168.113.6:9984'
bdb = BigchainDB(bdb_root_url)

# 提交事务
def transactionsPost(dict_data):
#     dict_data = {
#     "operation":"TRANSFER",
#     "asset":{"id":"123456"},
#     "signers"="admin_public_key"
#     "inputs":[{"input":"input_msg"}],
#     "recipients": [[["public_key1"],2],[["public_key2",6]]],
#     "metadata": {"a":1},
#     "private_keys": ["p1","p2"]
#     }

    tuple_recipients = []
    for i in dict_data["recipients"]:
        new_data = (i[0],i[1])
        tuple_recipients.append(new_data)
        
    # 准备事务
    prepared_transfer_tx = ""
    #  operation='CREATE',
    if dict_data["operation"] == 'CREATE':
        prepared_transfer_tx = bdb.transactions.prepare(
         operation=dict_data["operation"],
         asset=dict_data["asset"],
         signers=dict_data["signers"],
         recipients=tuple_recipients,
         metadata=dict_data["metadata"]

    )
    #  operation='TRANSFER',
    if dict_data["operation"] == 'TRANSFER':
        prepared_transfer_tx = bdb.transactions.prepare(
             operation=dict_data["operation"],
             asset=dict_data["asset"],
             inputs=dict_data["inputs"],
             recipients=tuple_recipients,
             metadata=dict_data["metadata"]

        )
    # 准备事务
    fulfilled_transfer_tx = bdb.transactions.fulfill(prepared_transfer_tx, private_keys=dict_data["private_keys"])
    # 提交事务
    sent_transfer_tx = bdb.transactions.send_commit(fulfilled_transfer_tx)
    # 验证提交结果
    if fulfilled_transfer_tx == sent_transfer_tx:
        print("事务提交成功")
        return fulfilled_transfer_tx
    return {"errMsg":"事务提交失败"}
    

# 生成密钥对
def generateKeypair():
    # generate a keypair
    keypair = generate_keypair()
    return {"publicKey": str(keypair.public_key),"privateKey": str(keypair.private_key)}

def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))


def on_message(client, userdata, msg):
    print(msg.topic + " " + str(msg.payload))
    
    reqTopic = str(msg.topic)
#     print(reqTopic, type(reqTopic))
    reqPayload = json.loads(msg.payload)  # 解码请求参数
    reqId = reqPayload["clientId"]
    transferData = reqPayload["transferData"]
    repTopic = reqTopic.replace("postServer", reqId)  # 生成响应topic
    
    if reqTopic == "postServer/secretKey":
        print(reqId+"生成密钥对")
        # 生成密钥对；
        print(json.dumps(generateKeypair()))
        client.publish(repTopic, payload=json.dumps(generateKeypair()), qos=0)
        return

    if reqTopic == "postServer/post":
        print("提交transaction")
        # chain 事务提交
        
        client.publish(repTopic, payload=json.dumps(transactionsPost(transferData)), qos=0)
        return
    
    print("undefine topic " + repTopic)
    return


client = mqtt.Client("postServer")
client.on_connect = on_connect
client.on_message = on_message
client.connect("192.168.113.6", 1883, 60)

client.subscribe('postServer/secretKey', qos=0)  # 请求密钥对
client.subscribe('postServer/post', qos=0)  # 提交transaction

# client.publish('block/emqtt', payload='Hello,EMQ!', qos=0)
client.loop_forever()


Connected with result code 0


KeyboardInterrupt: 

In [2]:
# test def generateKeypair()
from bigchaindb_driver.crypto import generate_keypair
import json

# 生成密钥对
def generateKeypair():
    # generate a keypair
    keypair = generate_keypair()
    return json.dumps({"publicKey": str(keypair.public_key),"privateKey": str(keypair.private_key)})

createKeypair()

'{"publicKey": "CiDPR8eWMWSBKBJWwB6aTFoShiTLiNC5hUPLHSNFjWNT", "privateKey": "CpoaYJKM8rr89tnGVd2NbykNAMfsuVz7iadmLvRbaKL4"}'

In [22]:
data = '''{"operation":"CREATE","asset":{"id":"","data":{"info":{"owner_nick_name":"block","owner_public_key":"3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm","type":"iot","id":"clock0","device_name":"clock0","device_info":"shareParking"},"sn":"3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm.iot.clock0"}},"signers":"3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm","inputs":0,"recipients":[[["3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm"],1]],"private_keys":["HwLCf9fbhm6BHTagY5aC1uVKR6sz57h7viuS8DUR9x34"],"metadata":{"info":{"device_id":"clock0","owner_nick_name":"block","owner_public_key":"3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm","user_nick_name":"null","user_public_key":"null","status":"CanUse","ruler":"5","start_time":"2019-05-13 04:12:49","cost_time":"0"},"sn":"3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm.iot.clock0"}}'''
# print(data)
dictData = json.loads(data)
print(dictData)


{'operation': 'CREATE', 'asset': {'id': '', 'data': {'info': {'owner_nick_name': 'block', 'owner_public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm', 'type': 'iot', 'id': 'clock0', 'device_name': 'clock0', 'device_info': 'shareParking'}, 'sn': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm.iot.clock0'}}, 'signers': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm', 'inputs': 0, 'recipients': [[['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'], 1]], 'private_keys': ['HwLCf9fbhm6BHTagY5aC1uVKR6sz57h7viuS8DUR9x34'], 'metadata': {'info': {'device_id': 'clock0', 'owner_nick_name': 'block', 'owner_public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm', 'user_nick_name': 'null', 'user_public_key': 'null', 'status': 'CanUse', 'ruler': '5', 'start_time': '2019-05-13 04:12:49', 'cost_time': '0'}, 'sn': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm.iot.clock0'}}


In [26]:
import json
from bigchaindb_driver import BigchainDB
bdb_root_url = 'http://192.168.113.6:9984'
bdb = BigchainDB(bdb_root_url)

# 提交事务
def transactionsPost(dict_data):
#     dict_data = {
#     "operation":"TRANSFER",
#     "asset":{"id":"123456"},
#     "signers"="admin_public_key"
#     "inputs":[{"input":"input_msg"}],
#     "recipients": [[["public_key1"],2],[["public_key2",6]]],
#     "metadata": {"a":1},
#     "private_keys": ["p1","p2"]
#     }

    tuple_recipients = []
    for i in dict_data["recipients"]:
        new_data = (i[0],i[1])
        tuple_recipients.append(new_data)
        
    print(tuple_recipients)
    # 准备事务
    prepared_transfer_tx = ""
    #  operation='CREATE',
    if dict_data["operation"] == 'CREATE':
        prepared_transfer_tx = bdb.transactions.prepare(
         operation=dict_data["operation"],
         asset=dict_data["asset"],
         signers=dict_data["signers"],
         recipients=tuple_recipients,
         metadata=dict_data["metadata"]

    )
    #  operation='TRANSFER',
    if dict_data["operation"] == 'TRANSFER':
        prepared_transfer_tx = bdb.transactions.prepare(
             operation=dict_data["operation"],
             asset=dict_data["asset"],
             inputs=dict_data["inputs"],
             recipients=tuple_recipients,
             metadata=dict_data["metadata"]

        )
    print(prepared_transfer_tx)
    
transactionsPost(dictData)

[(['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'], 1)]
{'inputs': [{'owners_before': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'], 'fulfills': None, 'fulfillment': {'type': 'ed25519-sha-256', 'public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'}}], 'outputs': [{'public_keys': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'], 'condition': {'details': {'type': 'ed25519-sha-256', 'public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'}, 'uri': 'ni:///sha-256;t1GM7Bud9-p2hvzkPDn8AN8FRSc8azG8u0BG_KLswiE?fpt=ed25519-sha-256&cost=131072'}, 'amount': '1'}], 'operation': 'CREATE', 'metadata': {'info': {'device_id': 'clock0', 'owner_nick_name': 'block', 'owner_public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm', 'user_nick_name': 'null', 'user_public_key': 'null', 'status': 'CanUse', 'ruler': '5', 'start_time': '2019-05-13 04:12:49', 'cost_time': '0'}, 'sn': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm.iot.clock0'}, 'asset': {'data': {'info': {'owner_nick_name': 'block